Based on: https://mc-stan.org/users/documentation/case-studies/pool-binary-trials.html

In [1]:
library(tidyverse)
library(yaml)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data_config_id <- "pooling_sim"

In [3]:
data_config <- yaml.load_file(paste0("../data/configs/", data_config_id, "/data.yaml"))

In [4]:
data_base_dir <- paste0("../", data_config$output_dir)
data_path <- paste0(data_base_dir, "/data.csv")
if (!dir.exists(data_base_dir)) {
    dir.create(data_base_dir, recursive = TRUE)
}

In [5]:
N_per_group <- data_config$N_per_group
N_groups <- data_config$N_groups
N_obs <- N_groups * N_per_group

beta_0 <- data_config$beta_0
beta_1 <- data_config$beta_1
sigma_group <- data_config$sigma_group
sigma_individual <- data_config$sigma_individual

min_x <- data_config$min_x
max_x <- data_config$max_x

data_seed <- data_config$data_seed

Our true data generating process

$y_{ij} = \beta_{0} + \beta_{1} X_{ij} + u_{0j} + e_{ij}$

Where:
- $y_{ij} \sim \mathcal{N}(\beta_{0} + \beta_{1} X_{ij} + u_{0j}, \sigma_{\text{student}}^2)$ = response variable individual $i$ in group $j$
- $u_{0j} \sim \mathcal{N}(0, \sigma_{\text{class}}^2)$ = the group-level effect for group $j$
- $e_{ij} \sim \mathcal{N}(0, \sigma_{\text{student}}^2)$ = the individual-level residual error term $i$
- $X_{ij}$ = observed measurement for individual $i$ in group $j$

## Simulate data

In [6]:
set.seed(data_seed)

In [7]:
data_df <- tibble(
        group_id = 1:N_groups,
        group_effect = rnorm(N_groups, 0, sigma_group)
    ) %>%
    inner_join(
        tibble(
            group_id = rep(1:N_groups, each=N_per_group),
            x = runif(N_obs, min = min_x, max = max_x),
            obs_error_term = rnorm(N_obs, mean = 0, sd = sigma_individual),
            obs_id = 1:N_obs
        ),
        by="group_id"
    ) %>%
    mutate(
        y = beta_0 + beta_1 * x + group_effect + obs_error_term
    )

In [8]:
data_df %>%
    write_csv(data_path)

In [9]:
data_df %>% pull(y) %>% quantile()

0%      25%      50%      75%     100% 
140.6385 172.6142 180.1298 189.6582 210.2446